Step1 : Create DP Workflow Template
Step2 : Configure active DP cluster(can alos configure new cluster)
Step3 : Add spark SQL or Pyspark Jobs to DP Workflow templates with Dependencies
Step4 : Run and Validate the DP Workflow Template

All the above steps can be executed using gcloud commands

Make sure to copy all the scripts and relevants files to GCS.   

In [ ]:
!gcloud config set dataproc/region us-central1

In [ ]:
!gcloud dataproc workflow-templates list

In [ ]:
#delete existing WF templates:

!gcloud dataproc workflow-templates delete template_name --quiet

In [ ]:
#create a new WF:
!gcloud dataproc workflow-templates create template_name
!gcloud dataproc workflow-templates list

In [ ]:
# Now configure your existing Dataproc cluster with your workflow:

!gcloud dataproc workflow-templates set-cluster-selector \
    wf_name \
        --cluster-labels goog-dataproc-cluster-name = cluster_name

#now run the commands to add jobs to the workflow(spark-sql is the type of job):

#cleanup job

gcloud dataproc workflow-templates add-job spark-sql \
    --step-id = job-cleanup \
    --file = gs://script_path
    --workflow-template= template_name


#File Format Convertor with Dependencies on cleanup:

gcloud dataproc workflow-templates add-job spark-sql \
    --step-id = job-convert-orders \
    --file = gs://script_path
    --params = bucket_name = gs://bucket_name, table_name =orders \
    --workflow-template= template_name \
    --start-after = job-cleanup

gcloud dataproc workflow-templates add-job spark-sql \
    --step-id = job-convert-order-items \
    --file = gs://script_path
    --params = bucket_name = gs://bucket_name, table_name =order_items \
    --workflow-template= template_name \
    --start-after = job-cleanup

#Job to compute  daily revenue:
gcloud dataproc workflow-templates add-job spark-sql \
    --step-id = job-daily-product-revenue \
    --file = gs://script_path
    --params = bucket_name = gs://bucket_name \
    --workflow-template= template_name \
    --start-after = job-convert-orders,job-convert-order-items

#job to read daily revenue data from gcs and write to GCP-BQ using pyspark(reformat the code to a single line if using windows):

gcloud dataproc workflow-templates add-job pyspark \
    --step-id = job-load-dpr-bq \
    --jars= gs://jarspath
    --properties=spark.app.name="Bigquery Loader -Daily Product Revenue",spark.submit.deployMode=cluster,spark.yarn.appMasterEnv.DATE_URI=gs://, define other environment variables \
    --workflow-template= template_name \
    --start-after = job-daily-product-revenue \
    gs://script_path


In [14]:
#To start the DP cluster use the below command:

!gcloud dataproc clusters list

NAME         PLATFORM  PRIMARY_WORKER_COUNT  SECONDARY_WORKER_COUNT  STATUS   ZONE           SCHEDULED_DELETE
abretaildev  GCE                                                     STOPPED  us-central1-f


In [ ]:
!gcloud dataproc clusters start cluster_name

In [ ]:
#Running the DP WF template:

!gcloud dataproc workflow-templates instantiate template_name

In [ ]:
# To remove a job from WF templatE:

!gcloud dataproc workflow-templates remove-job template_name --step-id = step-id   --quiet